In [1]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')

In [2]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'

In [3]:
modality_sestamibi = 'Sestamibi'
modality_4dct = '4D CT Scan'

preop_imaging_df['4D CT Imaging Side'] = preop_imaging_df[modality_4dct].apply(extract_side)
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

negative_sestamibi = preop_imaging_df[preop_imaging_df[modality_sestamibi] == 'Non-localizing'].copy()

sestamibi_neg_4d_pos = negative_sestamibi[negative_sestamibi['4D CT Imaging Side'].isin(['Left', 'Right'])]


In [4]:
def check_concordance(row):
    imaging = row['4D CT Scan']
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'

    
    

In [5]:
sestamibi_neg_4d_pos = sestamibi_neg_4d_pos.copy()
sestamibi_neg_4d_pos['Calculated Concordance'] = sestamibi_neg_4d_pos.apply(check_concordance, axis=1)

# total cases 
total_cases = sestamibi_neg_4d_pos.shape[0]

In [6]:
exact_count = sestamibi_neg_4d_pos[sestamibi_neg_4d_pos['Calculated Concordance'] == 'Yes (Exact)'].shape[0]
side_count = sestamibi_neg_4d_pos[sestamibi_neg_4d_pos['Calculated Concordance'] == 'Yes (Side)'].shape[0]
incorrect_count = sestamibi_neg_4d_pos[sestamibi_neg_4d_pos['Calculated Concordance'] == 'Incorrect Localization'].shape[0]
print(f'Exact Count: {exact_count}')
print(f'Side Count: {side_count}')
print(f'Incorrect Count: {incorrect_count}')

Exact Count: 10
Side Count: 6
Incorrect Count: 6


In [7]:
exact_pct = round((exact_count / total_cases) * 100, 2) if total_cases > 0 else None 
side_pct = round((side_count / total_cases) * 100, 2) if total_cases > 0 else None
incorrect_pct = round((incorrect_count / total_cases) * 100, 2) if total_cases > 0 else None 
print(f'Exact Percentage: {exact_pct}')
print(f'Side Percentage: {side_pct}')
print(f'Incorrect Percentage: {incorrect_pct}')

Exact Percentage: 45.45
Side Percentage: 27.27
Incorrect Percentage: 27.27


In [8]:
total_side_match_count = side_count + exact_count
total_side_match_pct = (total_side_match_count / total_cases) * 100 if total_cases > 0 else None

sestamibi_neg_4d_pos_summary_df = pd.DataFrame({
    'Match Type': [
        'Exact Match',
        'Total Side Match (Exact + Side-only)',
        'Incorect Localization'
    ],
    'Count': [
        exact_count,
        total_side_match_count,
        incorrect_count
    ],
    'Percentage': [
        f"{exact_pct:.2f}%" if exact_pct is not None else 'N/A',
        f"{total_side_match_pct:.2f}%" if total_side_match_pct is not None else 'N/A',
        f"{incorrect_pct:.2f}%" if incorrect_pct is not None else 'N/A'

    ]
     
})
print(f"Total Sestamibi negative / 4D CT positive cases: {total_cases}")
display(sestamibi_neg_4d_pos_summary_df)

Total Sestamibi negative / 4D CT positive cases: 22


,Match Type,Count,Percentage
0,Exact Match,10,45.45%
1,Total Side Match (Exact + Side-only),16,72.73%
2,Incorect Localization,6,27.27%


In [9]:
sestamibi_neg_4d_pos['Surgical_Findings'].value_counts()

Surgical_Findings
Left Superior                    5
Right Superior                   5
Right Inferior                   4
Left Inferior                    3
4 Gland Hyperplasia              2
Left, Right Superior             1
No Tumor Identified              1
Left Superior, Right Superior    1
Name: count, dtype: int64